<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/VGG16_transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

fatal: destination path 'AgeCalculation' already exists and is not an empty directory.


Age Estimation for Indian Movie Face Dataset

In [2]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications import vgg16
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd




In [3]:
data=pd.read_csv("/content/AgeCalculation/dataset/train.csv")

image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [4]:
data.shape

(19906, 2)

In [5]:
data.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [6]:
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
image_size = (224, 224)

In [7]:
#pretrained_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
#pretrained_resnet.summary()

In [9]:
pretrained_nn.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
dataset_path="/content/AgeCalculation/dataset/Train"

In [11]:
labels=[]
images_lst=[]
for image_name1,label in zip(image_name[:3000],label_name[:3000]):
	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+"/"+image_name1)
	labels.append(label)	#print(image_path)


	img = load_img(image_path, target_size=image_size)
	x = img_to_array(img)
	images_lst.append(x)




the image name  377.jpg and labelMIDDLE
the image name  17814.jpg and labelYOUNG
the image name  21283.jpg and labelMIDDLE
the image name  16496.jpg and labelYOUNG
the image name  4487.jpg and labelMIDDLE
the image name  6283.jpg and labelMIDDLE
the image name  23495.jpg and labelYOUNG
the image name  7100.jpg and labelYOUNG
the image name  6028.jpg and labelYOUNG
the image name  22617.jpg and labelOLD
the image name  11177.jpg and labelYOUNG
the image name  2462.jpg and labelMIDDLE
the image name  24116.jpg and labelMIDDLE
the image name  17689.jpg and labelMIDDLE
the image name  728.jpg and labelMIDDLE
the image name  3003.jpg and labelMIDDLE
the image name  14408.jpg and labelOLD
the image name  6679.jpg and labelYOUNG
the image name  15152.jpg and labelOLD
the image name  24784.jpg and labelMIDDLE
the image name  9970.jpg and labelYOUNG
the image name  22550.jpg and labelOLD
the image name  150.jpg and labelYOUNG
the image name  7379.jpg and labelMIDDLE
the image name  15387.jpg an

In [19]:
# Create a single numpy array with all the images we loaded
x_train = np.array(images_lst)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
#model = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features for each image (all in one pass)

features_x = pretrained_nn.predict(x_train)
#features_x = pretrained_resnet.predict(x_train)




94/94 [==============================] - 27s 158ms/step


In [18]:
x_train.shape

(3000, 224, 224, 3)

In [20]:
features_x.shape

(3000, 7, 7, 512)

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features_x, y_train, test_size=0.20, random_state=42)



In [23]:
# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)



In [26]:
# encode the labels using LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
from keras.utils import to_categorical
y_train = to_categorical(y_train, 3)

In [28]:
y_test = le.fit_transform(y_test)
y_test = to_categorical(y_test, 3)

In [29]:
#class_names = ["not_glaucoma","glaucoma"]
# Train the model
H = model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=50,shuffle=True)

Epoch 1/50
75/75 [==============================] - 3s 10ms/step - loss: 3.3297 - accuracy: 0.5054 - val_loss: 1.0386 - val_accuracy: 0.6417
Epoch 2/50
75/75 [==============================] - 1s 8ms/step - loss: 1.0776 - accuracy: 0.6679 - val_loss: 1.0938 - val_accuracy: 0.6250
Epoch 3/50
75/75 [==============================] - 0s 6ms/step - loss: 0.7672 - accuracy: 0.7300 - val_loss: 1.1547 - val_accuracy: 0.6183
Epoch 4/50
75/75 [==============================] - 0s 6ms/step - loss: 0.5592 - accuracy: 0.8087 - val_loss: 1.0419 - val_accuracy: 0.6867
Epoch 5/50
75/75 [==============================] - 1s 7ms/step - loss: 0.4863 - accuracy: 0.8379 - val_loss: 1.0072 - val_accuracy: 0.6583
Epoch 6/50
75/75 [==============================] - 1s 7ms/step - loss: 0.3292 - accuracy: 0.8871 - val_loss: 1.2307 - val_accuracy: 0.6567
Epoch 7/50
75/75 [==============================] - 1s 7ms/step - loss: 0.3098 - accuracy: 0.8963 - val_loss: 1.3222 - val_accuracy: 0.6483
Epoch 8/50
75/75 [=